# OperatorAware
## Urgency detection for emergency calls
mitchellpkt@protonmail.com

OperatorAware evaluates 911 calls in real-time to assess the nature and direness of the emergency being reported.

As the human operator collects information, OperatorAware converts the audio stream to text, and scans for keywords indicating critical situations (e.g. the word "gun" indicating that there is likely a weapon involved). As the call continues, OperatorAware updates the likelihood of a situation involving:
-  Weapons (e.g. inferred from use of words "gun" and "shot" by a caller)
-  Medical emergencies 
-  Vehicle accidents
-  etc

## Example
Caller reports: **"My neighbor shot my leg, It's bleeding everywhere! He didn't mean to discharge the gun"** 

OperatorAware uses the words {"shot", "bleeding", "gun"} to identify that this is a **medical emergency involving a weapon.**

## Motivation
There have been multiple reported instances of emergency operators hanging up on 911 callers during crises. 

For example, one operator who stated "You could deal with it yourself. I’m not gonna deal with this, okay?” before hanging up on a caller that was performing CPR on a shooting victim ([Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2015/07/29/deal-with-it-yourself-911-dispatcher-tells-panicked-caller-with-dying-friend/?utm_term=.eea24de1e5f3)). 

Another 911 dispatcher, infamous for ending a call with "Ain't nobody got time for this," hung up on reports of street racing and armerd robberies (different [Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2018/04/19/911-dispatcher-jailed-houston-woman-hung-up-on-thousands-of-callers/?noredirect=on&utm_term=.b4bb2b6e8f37) article). In this case, later analysis showed that thousands of calls shorter than 20 seconds were attributed to her hanging up. She stated that she hung up because at those times she did not want to talk to anyone ([BBC](http://www.bbc.com/news/world-us-canada-43822504)).

While 911 callers may hang up at any time, in the vast majority of cases, **the 911 dispatcher should not be the party to terminate the call during a crisis.** If an operator is the party to end a call that has been ranked as likely-severe by the OperatorAware, this should trigger quality assurance review. The latter dispatcher mentioned above hung up on thousands of calls over multiple years. With OperatorAware, she would have come under close scrutiny by the end of her first week

## Use case scale
The [Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2018/04/19/911-dispatcher-jailed-houston-woman-hung-up-on-thousands-of-callers/?noredirect=on&utm_term=.d2fbe079869a) notes: *This single consolidated center for 911 calls opened in 2003 and handles millions of calls every year, according to the Chronicle, or 9,000 a day. Two-thirds of those calls aren’t true emergencies*

*The rest involve people in dire need.*

## Changelog
-  v1, local processing using PocketSphinx
-  v2, trying 2 different implementations of Google voice-to-text (Inst 2 works!)
-  v3, cleaning up to the one that works, moving forward


# Preparation

## Enter name of file to be transcribed

In [1]:
sound_filename = 'fire_short'

## Select dictionary

In [2]:
# proof-of-concept dictionary. For actual implementation, would need multiple formes e.g. 'shoot' and 'shot'
DangerWords = {'weaponWords': ['knife', 'gun', 'weapon', 'shoot', 'shot', 'armed', 'shotgun', 'handgun'],
        'medicalWords': ['heart','stroke','breathing','unconscious','collapsed'],
        'vehicleWords': ['crash', 'accident','airbag'],
        'domesticWords': ['domestic','abuse','fight','argument','arguing','relationship'],
        'fireWords': ['fire','burn','arson'],
        'miscWords': ['violent','suicidal', 'suicide','drunk']};

DangerDefs = {'weaponWords': 'weapon',
        'medicalWords': 'medical emergency',
        'vehicleWords': 'car crash',
        'domesticWords': 'domestic altercation',
        'fireWords': 'fire',
        'miscWords': 'miscellaneuos'};

## Import/install libraries

In [3]:
#!pip install --upgrade google-cloud-speech

import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

## Load in the audio file

In [6]:
### Form filename
file_name = os.path.join(
     os.getcwd(),
    'Real_911Calls',
    sound_filename + '.flac')

### Load the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    #encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
    language_code='en-US')


# TEMP: NOODLE AROUND WITH HASHES

In [10]:
import hashlib

In [26]:
audio.SerializePartialToString()

b'\n\xe5\xb1\x18fLaC\x00\x00\x00"\x10\x00\x10\x00\x00\x0b\xbe\x00\x0f~\n\xc4@\xf0\x00\x07\x1c\xf0\xeb\x7f\xc7\xafz\xe9\xd6\xf3\x82R\xe6\xb33\x82x\x95\x84\x00\x00( \x00\x00\x00reference libFLAC 1.3.2 20170101\x00\x00\x00\x00\xff\xf8\xc9\x08\x00\x95L\x00D\x00\x0c\xff\xf3\xff\xd0\xff\x89\xff\x88\xffy\xb5\xaby\xa5~N\xbc\x04OA\x90\x88j\xc9\xf0\x88\xc8\x92\xfb\x88`bag\xa7\x97\x12\xfe\xd8\x8d[\x828\x96G/!\xaeS(\xd1\x85H\xb4\xa8\x81-\rxh/r\xd1qr\xe1Q\xc2\xb9\xbdY}j\xdd\xc4\xd7\xb2\xb6}\xa1\n0\xc5\x14\nC\x16w%([\x9c\xce\xae6\xdd\xc4\x04\xd4\xfc\xd4\xa4\x9f\xc0\xc0\x1bq\x15\x82\r\x9a3Z\xbcJ\xd6y\xd1\xbe\x12T\x85_Ox1.\xdd\x99Ih{\x85e\xa2IG\x96X)\xa3bT\xb6:~\xad\x0e\xbbi!\xa1\xb1\x03\xc2}\x07w\x18\xe5\xed\x93z\xb3x\x0b,\xc9R\xb5\xa7\xe8\xff\xa5\xe5\x07\x8b\xa4\x84\x87\xe4\xa6+\xc5&\xfd\xaaX\x05\xce\xebL\x8b\xe2S\xdbed\xb4\xaf\xc4*\xa1\xad$\xf7\xa6\xf5,\x13\x93\xba\xbe46\xa8\x0ed_\x82E|\xa0t_$\xaeY\xed_0\x0f\xa1TI\x12~\x8d\xc9l\xd9\x93m\xc3hd]2\x82g\xe47=\x88\x80k\xb72\xa8\xf5\x163b\x85\x8f0\xe1\x1

In [30]:
hash_object = hashlib.sha256(audio.SerializePartialToString())
hex_dig = hash_object.hexdigest()
print(hex_dig)

070164a969c89fc5c85a96c74cfb1abec170f744ca63f6b3475debb9d484b83e


## Instantiate a client

In [5]:
!export GOOGLE_APPLICATION_CREDENTIALS="/home/m/Dropbox/Projects/OperatorAware/GitPath/operatoraware/OperatorAware-5f653aaf3399.json"
client = speech.SpeechClient()

/home/m/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Has this call been transcribed?

In [10]:
# Where would/should the transcript file be stored

transcription_filename = os.path.join(
     os.getcwd(),
    'transcriptions',
    sound_filename+'.txt')

does_transcription_exist = os.path.isfile(transcription_filename)

## Process or import the call

In [12]:
if does_transcription_exist == True:
    with open(transcription_filename,'r') as f_open:
        TranscriptionString = f_open.read()
        
    print('***************************')
    print('Imported data from:' + transcription_filename)
    print('Transcription:')
    print(TranscriptionString)

else:
    # Detect speech in the audio file
    response = client.recognize(config, audio)

    TranscriptionString = ''
    for result in response.results:
        #print('Transcript: {}'.format(result.alternatives[0].transcript))
        TranscriptionString += ' ' + format(result.alternatives[0].transcript)
        
    with open(transcription_filename,'w') as f_open:
        f_open.write(TranscriptionString)
        f_open.close()
    
    print('***************************')
    print('Fresh transcription stored in: '+ transcription_filename)
    print('Transcription:')
    print(TranscriptionString)
        

***************************
Fresh transcription stored in: /home/m/Dropbox/Projects/OperatorAware/GitPath/operatoraware/transcriptions/fire_short.txt
Transcription:
 hello I am calling to report a fire my flask application is going to burn


## Display transcription

In [13]:
print(TranscriptionString)

 hello I am calling to report a fire my flask application is going to burn


## Search transcription for keywords

In [14]:
HeardWords = []
is_anything_important = 0

for danger_cat in DangerWords.keys():
    ThisCategory = DangerWords[danger_cat]
    for keyword in ThisCategory:
        if keyword in TranscriptionString:
            is_anything_important = 1
            print('Possible ' + DangerDefs[danger_cat] + ': ' + keyword)

## Classify call

In [15]:
if is_anything_important == 1:
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print('Likely emergency; operator should not terminate')
else:
    print('---------------------')
    print('No emergency detected')

---------------------
No emergency detected
